In [11]:
import xarray as xr
import netCDF4 as nc
import os
import datetime
from glob import glob
from calendar import isleap
import numpy as np

In [12]:
jradir = '/leonardo_scratch/fast/ICT24_MHPC/ntilinin/INPUT/OMIP/'
#variables = ['huss']
variables = ['friver', 'tas', 'huss', 'uas', 'vas', 'licalvf']
#variables = ['rsds', 'rlds', 'prra', 'prsn', 'psl']
#years = (1999, 2000)
FillValue = -9.99e34
variables

['friver', 'tas', 'huss', 'uas', 'vas', 'licalvf']

In [13]:
#for year1 in years:
year1 = 1990
year2 = year1 + 1

    # Define base year and time slices based on leap year
#if isleap(year2):
#    baseyear = year1
#    timeslice1 = slice(datetime.datetime(year1, 1, 1, 0, 0), datetime.datetime(year1, 4, 30, 23, 59))
#    timeslice2 = slice(datetime.datetime(year2, 1, 1, 0, 0), datetime.datetime(year2, 4, 29, 23, 59))
#    print(f"year2 is {year2}; year1 is {year1}; baseyear is {baseyear}, IS LEAP")   
#else:
baseyear = year2
timeslice1 = slice(datetime.datetime(year1, 5, 1, 0, 0), None)
timeslice2 = slice(datetime.datetime(year2, 5, 1, 0, 0), None)
ds = {}

In [14]:
for var in variables:
    for y in (year1,year2):
        files = glob(os.path.join(jradir, f"{var}*{y}*.nc"))
        ds[y] = xr.open_dataset(files[0], decode_coords=False)
    
    ryf = ds[baseyear].drop_vars("time_bnds", errors='ignore')
    ryf.encoding = ds[baseyear].encoding
    
    for varname in ryf.data_vars:
        if '_FillValue' not in ryf[varname].encoding: ryf[varname].encoding['_FillValue'] = FillValue
        
        if len(ryf[varname].shape) < 3: continue
        print(f"Processing variable data for {varname}")
        if isleap(year2):
            # Set the Jan->Apr values to those from the first year
            ryf[varname].loc[dict(time=timeslice1)] = ds[year2][varname].sel(time=timeslice2).values
        else:
            # Set the May->Dec values to those from the first year
            ryf[varname].loc[dict(time=timeslice2)] = ds[year1][varname].sel(time=timeslice1).values
    
    # Set FillValue for dimensions
    for dim in ryf.dims:
     # Have to give all dimensions a useless FillValue attribute, otherwise xarray
     # makes it NaN and MOM does not like this
        ryf[dim].encoding['_FillValue'] = FillValue
    # Make a new time dimension with no offset from origin (1900-01-01) so we don't get an offset after
        # changing calendar to noleap
    newtime = (ryf.indexes["time"].values - ryf.indexes["time"].values[0]) + np.datetime64('1900-01-01','D')
    ryf.indexes["time"].values[:] = newtime[:]

    ryf["time"].attrs = {'modulo':' ',
                    'axis':'T',
                    'cartesian_axis':'T',
                    # 'calendar':'noleap'
    }

    outfile = "RYF.{}.{}_{}.nc".format(var,year1,year2)
    ryf.to_netcdf(outfile)    
    # Open the file again directly with the netCDF4 library to change the calendar attribute. xarray
    # has a hissy fit as this violates it's idea of CF encoding if it is done before writing the file above
    ryf = nc.Dataset(outfile, mode="r+")
    ryf.variables["time"].calendar = "noleap"
    ryf.close()      
    del ryf

Processing variable data for friver
Processing variable data for tas
Processing variable data for huss
Processing variable data for uas
Processing variable data for vas
Processing variable data for licalvf
